In [23]:
from numpy import hstack, vstack, array, median, nan
from numpy.random import choice
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

### load data

In [17]:
#特征矩阵加工
#使用vstack增加一行含缺失值的样本(nan, nan, nan, nan)
#使用hstack增加一列表示花的颜色（0-白、1-黄、2-红），花的颜色是随机的，意味着颜色并不影响花的分类

iris = load_iris()
iris.data = hstack((choice([0, 1, 2], size=iris.data.shape[0]+1).reshape(-1,1), vstack((iris.data, array([nan, nan, nan, nan]).reshape(1,-1)))))

In [18]:
iris.data

array([[1. , 5.1, 3.5, 1.4, 0.2],
       [0. , 4.9, 3. , 1.4, 0.2],
       [2. , 4.7, 3.2, 1.3, 0.2],
       [0. , 4.6, 3.1, 1.5, 0.2],
       [1. , 5. , 3.6, 1.4, 0.2],
       [0. , 5.4, 3.9, 1.7, 0.4],
       [2. , 4.6, 3.4, 1.4, 0.3],
       [1. , 5. , 3.4, 1.5, 0.2],
       [1. , 4.4, 2.9, 1.4, 0.2],
       [1. , 4.9, 3.1, 1.5, 0.1],
       [0. , 5.4, 3.7, 1.5, 0.2],
       [2. , 4.8, 3.4, 1.6, 0.2],
       [2. , 4.8, 3. , 1.4, 0.1],
       [0. , 4.3, 3. , 1.1, 0.1],
       [1. , 5.8, 4. , 1.2, 0.2],
       [1. , 5.7, 4.4, 1.5, 0.4],
       [1. , 5.4, 3.9, 1.3, 0.4],
       [2. , 5.1, 3.5, 1.4, 0.3],
       [0. , 5.7, 3.8, 1.7, 0.3],
       [2. , 5.1, 3.8, 1.5, 0.3],
       [0. , 5.4, 3.4, 1.7, 0.2],
       [1. , 5.1, 3.7, 1.5, 0.4],
       [2. , 4.6, 3.6, 1. , 0.2],
       [2. , 5.1, 3.3, 1.7, 0.5],
       [1. , 4.8, 3.4, 1.9, 0.2],
       [1. , 5. , 3. , 1.6, 0.2],
       [2. , 5. , 3.4, 1.6, 0.4],
       [2. , 5.2, 3.5, 1.5, 0.2],
       [2. , 5.2, 3.4, 1.4, 0.2],
       [2. , 4

In [21]:
# 目标值向量加工
# 增加一个目标值，对应含缺失值的样本，值为众数
iris.target = hstack((iris.target, array([median(iris.target)])))
iris.target

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 2., 1., 1.])

In [26]:
np.log1p(1)

0.6931471805599453

In [27]:
np.log(2)

0.6931471805599453

### 整体并行处理

In [29]:
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import Binarizer
from sklearn.pipeline import FeatureUnion

In [31]:
# 新建将整体特征进行对数函数转换的对象
# FunctionTransformer将一个python函数转变成transformer
step2_1 = ('ToLog', FunctionTransformer(np.log1p))
step2_1

('ToLog', FunctionTransformer(accept_sparse=False, func=<ufunc 'log1p'>,
           inv_kw_args=None, inverse_func=None, kw_args=None,
           pass_y='deprecated', validate=True))

In [33]:
step2_2 = ('ToBinary', Binarizer())
step2_2

('ToBinary', Binarizer(copy=True, threshold=0.0))

In [40]:
# 新建整体并行处理对象。该对象也有fit和transform方法
step2 = ('FeatureUnion', FeatureUnion(transformer_list=[step2_1, step2_2]))
step2


('FeatureUnion', FeatureUnion(n_jobs=1,
        transformer_list=[('ToLog', FunctionTransformer(accept_sparse=False, func=<ufunc 'log1p'>,
           inv_kw_args=None, inverse_func=None, kw_args=None,
           pass_y='deprecated', validate=True)), ('ToBinary', Binarizer(copy=True, threshold=0.0))],
        transformer_weights=None))

In [41]:
step3 = FeatureUnion(transformer_list=[step2_1, step2_2])
step3

FeatureUnion(n_jobs=1,
       transformer_list=[('ToLog', FunctionTransformer(accept_sparse=False, func=<ufunc 'log1p'>,
          inv_kw_args=None, inverse_func=None, kw_args=None,
          pass_y='deprecated', validate=True)), ('ToBinary', Binarizer(copy=True, threshold=0.0))],
       transformer_weights=None)

In [39]:
step3.fit_transform(iris.data)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [45]:
from sklearn.preprocessing import Imputer
Imputer.

In [44]:
# 计算处理缺失值

from sklearn.impute import SimpleImputer
imp = SimpleImputer(missing_values=np.nan, strategy='mean')
imp

ModuleNotFoundError: No module named 'sklearn.impute'